In [243]:
import pandas as pd

data = pd.read_csv (r"/Users/silviaserafini/Desktop/attacks.csv",encoding='latin-1')
df = pd.DataFrame(data)
display(df.head())
print(df.shape)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


(25723, 24)


# General inspection-operations

In [244]:
#inspection of the names of the columns
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [245]:
#dropping unusefull columns
df=df.drop(['Case Number','Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],axis=1)

#dropping unusefull rows    
df_clean=df.dropna(axis=0, how='all', thresh=None, subset=None, inplace=False)

#removing of eventual duplicates
df_clean=df_clean.drop_duplicates()


In [246]:

print(df_clean.shape)

(6300, 14)


In [247]:
# inspection of the column "Fatal (Y/N), to see the % of null values"
nulle=df_clean["Fatal (Y/N)"].isnull().sum()
notNulle=df_clean["Fatal (Y/N)"].count()
perc=notNulle/(nulle+notNulle)
print(perc)
print(nulle)
print(notNulle)
print(nulle+notNulle)

0.914444444444
539
5761
6300


# Cleaning of 'Year' column

In [248]:
#trying to fill in the empty cells of the 'Year' column by inserting the year found with a regex in the same row, Date column

#1-definition of a function to apply to series 'Date' to extract the year
import re

def fn(txt):
    txt=str(txt)
    if re.findall(r'[0-9]{4}',txt):
        return re.findall(r'[0-9]{4}',txt)[0]
    else :
        return 'unknown'

#2-Storage of the guessed year on a temporary support column 'DataTent'                                                                                                                            
df_clean['DateTent']=df_clean['Date'].apply(fn)

#3-replace the value of DataTent in Year isnull cells
df_clean.loc[df_clean['Year'].isnull(),"Year"] = df_clean['DateTent'][df_clean['Year'].isnull()]

#4-replace value Year == 0 or Year==unknown
for i in range(df_clean.shape[0]):
    if df_clean.iloc[i][2]== '0' or df_clean.iloc[i][2]== 'unknown':
        df_clean.iloc[i][2]= df_clean.iloc[i][15]                                                 

#5-value check of column year: keep only valid years
df_clean['Year']=df_clean['Year'].apply(fn)                                           

#6-hand change of 2 values of cases happened during the second world war:
df_clean.loc[df_clean["Year"].str.startswith("W"),"Year"] = "1939"                                                          

#7-Drop dataTent column
df_clean=df_clean.drop('DateTent', axis=1)



In [249]:
#8-drop the last rows with unknown year. These cases are very old and not relevant for the analysis 


filter1 = []
for result in df_clean.Year:
    if not re.search('unknown', result):
        filter1.append(True)
    else:
        filter1.append(False)

#display(df_clean[filter1])

df_clean=df_clean[filter1]
print(df_clean.Year.sort_values().value_counts())



2015    143
2017    137
2016    130
2011    128
2014    127
2008    122
2013    122
2009    120
2012    117
2007    112
2005    103
2006    103
2010    101
2000     97
1959     93
1960     93
2004     92
2003     92
2001     92
2002     88
1962     86
1961     78
1995     76
1999     66
1964     66
1998     65
1996     61
1963     61
1966     58
1997     57
       ... 
1638      1
1784      1
1791      1
1783      1
1543      1
1767      1
1804      1
1792      1
1748      1
1859      1
1807      1
1742      1
1857      1
1802      1
1755      1
1801      1
1723      1
1555      1
1787      1
1810      1
1721      1
1779      1
1733      1
1554      1
1797      1
1749      1
1703      1
1637      1
1738      1
1764      1
Name: Year, Length: 245, dtype: int64


In [250]:
#sort of the database according to the column 'Year'
df_clean=df_clean.sort_values(by=['Year','Date'], ascending=False, na_position='first')

# Cleaning of 'Type' column

In [251]:

print(df_clean.isnull().sum())
df_clean.Type.value_counts()

Date              0
Year              0
Type              4
Country          45
Area            428
Location        504
Activity        524
Name            205
Sex             557
Age            2714
Injury           27
Fatal (Y/N)     538
Time           3231
Species        2744
dtype: int64


Unprovoked      4482
Provoked         567
Invalid          545
Sea Disaster     233
Boating          203
Boat             135
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [252]:
#inspection: what does 'invalid means?'
df_clean.loc[df_clean.Type=='Invalid']

#I can drop these rows, since for these entries sharks involvement is not confirmed 
df_clean=df_clean.loc[(df_clean.Type!='Invalid') & (df_clean.Type!='Questionable')]

#recategorization
df_clean['Type']=df_clean['Type'].replace(['Boat','Boatomg'],'Boating')


In [253]:
#recategorization
df_clean['Type']=df_clean['Type'].replace('Sea Disaster','Unprovoked')


In [254]:
#replacement of the null values with the mostfrequent category value
df_clean['Type'].isnull().sum()
df_clean[df_clean['Type'].isnull()]
df_clean['Type'].describe()
df_clean[df_clean['Type'].isnull()]=df_clean[df_clean['Type'].isnull()].fillna('Unprovoked')
df_clean['Type'].isnull().sum()

0

# Claning of 'Fatal (Y/N)' column

In [255]:
#claning of 'Fatal (Y/N)' column
df_clean.head()
df_clean['Fatal (Y/N)'].isnull().sum()


14

In [256]:
#Definition of a function to clean 'Fatal (Y/N)'. 
#Label 'Y' if in the description It finds the words 'FATAL' or 'drowning'

def fn1(x):
    if re.findall('FATAL',str(x).upper()) or re.findall('drowning',str(x)) :
        return 'Y'
    else:
        return 'N'
    
res=df_clean.loc[df_clean['Fatal (Y/N)'].isnull()]['Fatal (Y/N)'].apply(fn1)
display(res)

27      N
294     N
269     N
1292    N
3452    N
3506    N
4127    N
4333    N
6125    N
5630    N
5945    N
6104    N
6122    N
6158    N
Name: Fatal (Y/N), dtype: object

In [257]:
#check
df_clean.head()


,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
17,Reported 30-Apr-2018,2018,Unprovoked,THAILAND,Hua Hin,Sai Noi Beach,Swimming,female,M,NaN,Minor injury to ankle,N,NaN,NaN
14,May 2018,2018,Provoked,AUSTRALIA,Westerm Australia,Dugong Bay,Feeding sharks,Melisa Brunning,F,34,Shallow lacerations to finger PROVOKED INCIDENT,N,NaN,"Tawny nurse shark, 2m"
37,9-Mar-2018,2018,Unprovoked,AUSTRALIA,Victoria,Lorne,Swimming,Rob Crossland,M,69,"No injury, said to have been charged by sharks...",N,15h30,3.5 m shark
38,9-Mar-2018,2018,Unprovoked,AUSTRALIA,Victoria,Winkipop,Surfing,Lachie Brown,M,18,"Minor injury, ankle grazed",N,07h30,1.8 m shark
35,31-Mar-2018,2018,Unprovoked,USA,Hawaii,Kukio Beach,Stand-Up Paddleboarding,male,M,25,Injuries to right leg & hand,N,09h30,"Tiger shark, 12'"


In [258]:
#check and replacing of the null values with 'unknown'
df_clean['Country'].value_counts()
df_clean['Country'].isnull().sum()
df_analysis=df_clean.fillna('unknown')
df_analysis.replace('unkown','unknown')

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
17,Reported 30-Apr-2018,2018,Unprovoked,THAILAND,Hua Hin,Sai Noi Beach,Swimming,female,M,unknown,Minor injury to ankle,N,unknown,unknown
14,May 2018,2018,Provoked,AUSTRALIA,Westerm Australia,Dugong Bay,Feeding sharks,Melisa Brunning,F,34,Shallow lacerations to finger PROVOKED INCIDENT,N,unknown,"Tawny nurse shark, 2m"
37,9-Mar-2018,2018,Unprovoked,AUSTRALIA,Victoria,Lorne,Swimming,Rob Crossland,M,69,"No injury, said to have been charged by sharks...",N,15h30,3.5 m shark
38,9-Mar-2018,2018,Unprovoked,AUSTRALIA,Victoria,Winkipop,Surfing,Lachie Brown,M,18,"Minor injury, ankle grazed",N,07h30,1.8 m shark
35,31-Mar-2018,2018,Unprovoked,USA,Hawaii,Kukio Beach,Stand-Up Paddleboarding,male,M,25,Injuries to right leg & hand,N,09h30,"Tiger shark, 12'"
47,28-Jan-2018,2018,Unprovoked,AUSTRALIA,Western Australia,Cone Bay,unknown,male,M,18,Injuries to right leg,N,08h00,Bull shark
18,28-Apr-2018,2018,Unprovoked,COSTA RICA,Cocos Island,Manuelita,Scuba diving,male,M,30,"No injury, shark bit scuba gear",N,10h40,Tiger shark
19,28-Apr-2018,2018,Unprovoked,AUSTRALIA,Western Australia,Gearys Beach,Surfing,Adam Murray,M,unknown,"No injury, knocked off board by shark",N,16h50,unknown
7,27-May-2018,2018,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,unknown,"Lemon shark, 3'"
8,26-May-2018,2018,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'"


In [259]:
#check
df_analysis.isnull().sum()

Date           0
Year           0
Type           0
Country        0
Area           0
Location       0
Activity       0
Name           0
Sex            0
Age            0
Injury         0
Fatal (Y/N)    0
Time           0
Species        0
dtype: int64

In [260]:
#transformation of 'Fatal (Y/N)' column in int value 1-0
df_analysis['Fatal (Y/N)']=[re.sub(r'unknown','0', str(el).strip().upper()) for el in df_analysis['Fatal (Y/N)']]
df_analysis['Fatal (Y/N)']=df_analysis['Fatal (Y/N)'].replace('UNKNOWN','unknown')
df_analysis['Fatal (Y/N)']=df_analysis['Fatal (Y/N)'].replace('UNPROVOKED','1')
df_analysis['Fatal (Y/N)']=[re.sub(r'[N\n]{1}','0', str(el).strip()) for el in df_analysis['Fatal (Y/N)']]
df_analysis['Fatal (Y/N)']=df_analysis['Fatal (Y/N)'].replace('Y','1')
df_analysis['Fatal (Y/N)']=df_analysis['Fatal (Y/N)'].replace('y','1')
df_analysis['Fatal (Y/N)']=df_analysis['Fatal (Y/N)'].replace('M','0')
df_analysis['Fatal (Y/N)']=df_analysis['Fatal (Y/N)'].replace('2017','1')
df_analysis['Fatal (Y/N)']=df_analysis['Fatal (Y/N)'].replace('U0PROVOKED','1')

df_analysis["Fatal (Y/N)"].value_counts()

0          4219
1          1324
unknown      82
Name: Fatal (Y/N), dtype: int64

In [261]:
#creation of 2 dummies variables for the value 'Fatal', 'NotFatal'. 
#These will be usefull for someof the plots of the analysis

df_analysis['Fatal']=df_analysis['Fatal (Y/N)']
df_analysis['NotFatal']=df_analysis['Fatal (Y/N)'].replace('0','2')
df_analysis['NotFatal']=df_analysis['NotFatal'].replace('1','0')
df_analysis['NotFatal']=df_analysis['NotFatal'].replace('2','1')
df_analysis.head()

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Fatal,NotFatal
17,Reported 30-Apr-2018,2018,Unprovoked,THAILAND,Hua Hin,Sai Noi Beach,Swimming,female,M,unknown,Minor injury to ankle,0,unknown,unknown,0,1
14,May 2018,2018,Provoked,AUSTRALIA,Westerm Australia,Dugong Bay,Feeding sharks,Melisa Brunning,F,34,Shallow lacerations to finger PROVOKED INCIDENT,0,unknown,"Tawny nurse shark, 2m",0,1
37,9-Mar-2018,2018,Unprovoked,AUSTRALIA,Victoria,Lorne,Swimming,Rob Crossland,M,69,"No injury, said to have been charged by sharks...",0,15h30,3.5 m shark,0,1
38,9-Mar-2018,2018,Unprovoked,AUSTRALIA,Victoria,Winkipop,Surfing,Lachie Brown,M,18,"Minor injury, ankle grazed",0,07h30,1.8 m shark,0,1
35,31-Mar-2018,2018,Unprovoked,USA,Hawaii,Kukio Beach,Stand-Up Paddleboarding,male,M,25,Injuries to right leg & hand,0,09h30,"Tiger shark, 12'",0,1


In [262]:
#Dropping of the anunused column 'Fatal (Y/N)'
df_analysis=df_analysis.drop('Fatal (Y/N)', axis=1)

# Cleaning of the 'Species' column 

In [263]:
#cleaning of the species column

df_analysis.columns

#1-the column name had an unwanted space
df_analysis['Species']=df_analysis['Species ']
df_analysis=df_analysis.drop('Species ',axis=1)

#2-inspection of the column
df_analysis['Species'].sample(50).value_counts()

#definition of a function to apply to each cell in the series, to extract the name of the species.
def fn2(x):
    x=str(x).upper()
    if len(re.findall(r'[A-Z]+\sSHARK',x))>0:
        return re.findall(r'[A-Z]+\sSHARK',x)[0].split('SHARK')[0].lower().strip()
    else:
        return 'unknown'

#application of the function on the series
df_analysis['Species1']=df_analysis['Species'].apply(fn2)



In [264]:
#I decided to keep only the real species with the length of the name longer than 2. 
#For all the other the specie is 'unknown'.

df_analysis['Species1']=df_analysis['Species1'].replace('m','unknown')
df_analysis['Species1'].value_counts()
df_analysis['Species1'] = pd.Series(['unknown' if len(str(el))<=2  else str(el)  for el in df_analysis['Species1'] ])

In [265]:
#check and dropping of temp column 'Species1'

df_analysis['Species1'].value_counts()
df_analysis['Species']=df_analysis['Species1']
df_analysis=df_analysis.drop('Species1',axis=1)

In [266]:
df_analysis.head()

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Time,Fatal,NotFatal,Species
17,Reported 30-Apr-2018,2018,Unprovoked,THAILAND,Hua Hin,Sai Noi Beach,Swimming,female,M,unknown,Minor injury to ankle,unknown,0,1,tiger
14,May 2018,2018,Provoked,AUSTRALIA,Westerm Australia,Dugong Bay,Feeding sharks,Melisa Brunning,F,34,Shallow lacerations to finger PROVOKED INCIDENT,unknown,0,1,blacktip
37,9-Mar-2018,2018,Unprovoked,AUSTRALIA,Victoria,Lorne,Swimming,Rob Crossland,M,69,"No injury, said to have been charged by sharks...",15h30,0,1,unknown
38,9-Mar-2018,2018,Unprovoked,AUSTRALIA,Victoria,Winkipop,Surfing,Lachie Brown,M,18,"Minor injury, ankle grazed",07h30,0,1,wobbegong
35,31-Mar-2018,2018,Unprovoked,USA,Hawaii,Kukio Beach,Stand-Up Paddleboarding,male,M,25,Injuries to right leg & hand,09h30,0,1,small


# Creation of a new 'Season' column

In [267]:
#Creation a new 'Season' column, staring from the column Date and extracting the month. 
#Then I recategorized the months with the superSet 'Season'

#1- definition of a function to extract the month from a Date entry
def fn3(x):
    x=str(x).upper()
    if len(re.findall(r'[A-Za-z]{3}-?',x))>0:
        return re.findall(r'[A-Za-z]{3}-?',x)[0].split('-')[0].lower().strip()
    else:
        return 'unknown'

#2-application of the function to the serie 'Date'    
df_analysis['Date']=df_analysis['Date'].apply(fn3)

#3-recategorization of the series with the Season labels
df_analysis['Date']=df_analysis['Date'].replace('rep','sep')
df_analysis['Date']=df_analysis['Date'].replace(to_replace=['dec','jan','feb','ter'], value ='win')
df_analysis['Date']=df_analysis['Date'].replace(to_replace=['sep','oct','nov','umn'], value ='aut')
df_analysis['Date']=df_analysis['Date'].replace(to_replace=['aug','jun','jul','mer'], value ='sum')
df_analysis['Date']=df_analysis['Date'].replace(to_replace=['may','mar','apr','ing'], value ='spr')

#4-all the entries with unknown months will be labelled with 'unk'
filter1=df_analysis['Date'].isin(['win','sum','aut','spr'])
filter1=~filter1
df_analysis.loc[filter1,'Date']='unk'

#5-check
df_analysis['Date'].value_counts()

#Dropping of the column 'Date', not usefull for my analysis
df_analysis['Season']=df_analysis['Date']
df_analysis=df_analysis.drop('Date',axis=1)

In [268]:
df_analysis.to_csv("/Users/silviaserafini/ironhack/PROGETTI/SharksData/OUTPUT/sharks_clean.csv")